In [1]:
from statsmodels.tsa.stattools import grangercausalitytests
import pandas as pd
import numpy as np

In [2]:
# 修改日期格式
def date_format(dt):
    dt = str(dt)
    year = dt[:dt.find("年")]
    month = dt[dt.find("年")+1:dt.find("月")]
    day = dt[dt.find("月")+1:dt.find("日")]
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    return year + '-' + month + '-' + day

# Top100

In [3]:
df = pd.read_excel("../相关性系数/日维度关键词热度排序总表.xlsx")
df = df.pivot_table(index='日期',columns='关键词',values='系数',aggfunc='sum').fillna(0)
df2 = pd.read_excel("../相关性系数/相关系数排名-上证综指.xlsx")
df = df[list(df2['关键词'][:100])]
df

关键词,口罩,武汉,湖北,开学,美国,病例,意大利,新冠,肺炎,礼裙,...,N95,基金,治愈,一站,神山,宋凛,男子,毕业生,死亡,泰国
日期,,,,,,,,,,,,,,,,,,,,,
2020-01-01,0.0,0.000301,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000831,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
2020-01-02,0.0,0.011621,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.022819,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.001084,0.000000,0.000000,0.0
2020-01-03,0.0,0.008366,0.0,0.0,0.010290,0.000000,0.0,0.000000,0.008366,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.005467,0.000741,0.000000,0.0
2020-01-04,0.0,0.002468,0.0,0.0,0.033193,0.000000,0.0,0.000000,0.002468,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.007692,0.0
2020-01-05,0.0,0.022405,0.0,0.0,0.017324,0.000000,0.0,0.000000,0.022405,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.008424,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,0.0,0.000000,0.0,0.0,0.007717,0.014999,0.0,0.005551,0.000000,0.000000,...,0.0,0.001100,0.003162,0.0,0.0,0.0,0.003849,0.000000,0.001842,0.0
2021-12-28,0.0,0.003299,0.0,0.0,0.015005,0.008207,0.0,0.009740,0.000000,0.000936,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.015079,0.008047,0.002355,0.0
2021-12-29,0.0,0.000000,0.0,0.0,0.006471,0.001915,0.0,0.000398,0.000000,0.000000,...,0.0,0.004056,0.000000,0.0,0.0,0.0,0.027498,0.000000,0.004343,0.0


In [4]:
df1 = pd.read_csv("../相关性系数/上证指数历史数据.csv")
df1['日期'] = df1['日期'].apply(date_format)
df1 = df1.sort_values(by='日期').set_index('日期')
df1 = df1['收盘']
df3 = pd.merge(df1, df, how='left',left_index=True,right_index=True)

In [5]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[['收盘', column_name]], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=12.3281 , p=0.0004  , df=1
likelihood ratio test: chi2=12.1741 , p=0.0005  , df=1
parameter F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.4973  , p=0.0001  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=19.1928 , p=0.0001  , df=2
likelihood ratio test: chi2=18.8221 , p=0.0001  , df=2
parameter F test:         F=9.4973  , p=0.0001  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=12.3281 , p=0.0004  , df=1
likelihood ratio test: chi2=12.1741 , p=0.0005  , df=1
parameter F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.4973  , p=0.0001  

ssr based F test:         F=0.2318  , p=0.6304  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.2332  , p=0.6292  , df=1
likelihood ratio test: chi2=0.2331  , p=0.6292  , df=1
parameter F test:         F=0.2318  , p=0.6304  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3099  , p=0.7337  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.6263  , p=0.7311  , df=2
likelihood ratio test: chi2=0.6259  , p=0.7313  , df=2
parameter F test:         F=0.3099  , p=0.7337  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2318  , p=0.6304  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.2332  , p=0.6292  , df=1
likelihood ratio test: chi2=0.2331  , p=0.6292  , df=1
parameter F test:         F=0.2318  , p=0.6304  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3099  , p=0.7337  , df_denom=479, df_num=2
ssr based chi2 test: 

parameter F test:         F=0.0357  , p=0.8503  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1004  , p=0.9045  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.2028  , p=0.9036  , df=2
likelihood ratio test: chi2=0.2028  , p=0.9036  , df=2
parameter F test:         F=0.1004  , p=0.9045  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0357  , p=0.8503  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.0359  , p=0.8497  , df=1
likelihood ratio test: chi2=0.0359  , p=0.8497  , df=1
parameter F test:         F=0.0357  , p=0.8503  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1004  , p=0.9045  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.2028  , p=0.9036  , df=2
likelihood ratio test: chi2=0.2028  , p=0.9036  , df=2
parameter F test:         F=0.1004  , p=0.9045  , df_denom=479, df_num=2

Granger Causality
nu

ssr based F test:         F=0.9689  , p=0.3255  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.9749  , p=0.3235  , df=1
likelihood ratio test: chi2=0.9739  , p=0.3237  , df=1
parameter F test:         F=0.9689  , p=0.3255  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5234  , p=0.5929  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.0577  , p=0.5893  , df=2
likelihood ratio test: chi2=1.0565  , p=0.5896  , df=2
parameter F test:         F=0.5234  , p=0.5929  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.9689  , p=0.3255  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.9749  , p=0.3235  , df=1
likelihood ratio test: chi2=0.9739  , p=0.3237  , df=1
parameter F test:         F=0.9689  , p=0.3255  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5234  , p=0.5929  , df_denom=479, df_num=2
ssr based chi2 test: 

likelihood ratio test: chi2=3.6884  , p=0.0548  , df=1
parameter F test:         F=3.6795  , p=0.0557  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.9971  , p=0.1369  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=4.0359  , p=0.1329  , df=2
likelihood ratio test: chi2=4.0192  , p=0.1340  , df=2
parameter F test:         F=1.9971  , p=0.1369  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.2270  , p=0.2685  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=1.2347  , p=0.2665  , df=1
likelihood ratio test: chi2=1.2331  , p=0.2668  , df=1
parameter F test:         F=1.2270  , p=0.2685  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.4179  , p=0.2432  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=2.8654  , p=0.2387  , df=2
likelihood ratio test: chi2=2.8570  , p=0.2397  , df=2
parameter F test:         F=1.4179  , p

...

likelihood ratio test: chi2=0.5834  , p=0.4450  , df=1
parameter F test:         F=0.5801  , p=0.4466  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5285  , p=0.5898  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.0681  , p=0.5862  , df=2
likelihood ratio test: chi2=1.0669  , p=0.5866  , df=2
parameter F test:         F=0.5285  , p=0.5898  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1443  , p=0.7042  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.1452  , p=0.7031  , df=1
likelihood ratio test: chi2=0.1452  , p=0.7032  , df=1
parameter F test:         F=0.1443  , p=0.7042  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0996  , p=0.9052  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.2013  , p=0.9042  , df=2
likelihood ratio test: chi2=0.2013  , p=0.9043  , df=2
parameter F test:         F=0.0996  , p


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.3263  , p=0.2500  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=1.3345  , p=0.2480  , df=1
likelihood ratio test: chi2=1.3327  , p=0.2483  , df=1
parameter F test:         F=1.3263  , p=0.2500  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.8174  , p=0.4422  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.6519  , p=0.4378  , df=2
likelihood ratio test: chi2=1.6491  , p=0.4384  , df=2
parameter F test:         F=0.8174  , p=0.4422  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.3263  , p=0.2500  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=1.3345  , p=0.2480  , df=1
likelihood ratio test: chi2=1.3327  , p=0.2483  , df=1
parameter F test:         F=1.3263  , p=0.2500  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.8174  , p=0.4422  

likelihood ratio test: chi2=0.1025  , p=0.7488  , df=1
parameter F test:         F=0.1019  , p=0.7497  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.0031  , p=0.3675  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=2.0272  , p=0.3629  , df=2
likelihood ratio test: chi2=2.0230  , p=0.3637  , df=2
parameter F test:         F=1.0031  , p=0.3675  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1019  , p=0.7497  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.1026  , p=0.7488  , df=1
likelihood ratio test: chi2=0.1025  , p=0.7488  , df=1
parameter F test:         F=0.1019  , p=0.7497  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.0031  , p=0.3675  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=2.0272  , p=0.3629  , df=2
likelihood ratio test: chi2=2.0230  , p=0.3637  , df=2
parameter F test:         F=1.0031  , p

ssr based F test:         F=4.6432  , p=0.0317  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=4.6721  , p=0.0307  , df=1
likelihood ratio test: chi2=4.6498  , p=0.0311  , df=1
parameter F test:         F=4.6432  , p=0.0317  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.0763  , p=0.0470  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=6.2168  , p=0.0447  , df=2
likelihood ratio test: chi2=6.1772  , p=0.0456  , df=2
parameter F test:         F=3.0763  , p=0.0470  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=4.6432  , p=0.0317  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=4.6721  , p=0.0307  , df=1
likelihood ratio test: chi2=4.6498  , p=0.0311  , df=1
parameter F test:         F=4.6432  , p=0.0317  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.0763  , p=0.0470  , df_denom=479, df_num=2
ssr based chi2 test: 

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1849  , p=0.6674  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.1861  , p=0.6662  , df=1
likelihood ratio test: chi2=0.1860  , p=0.6662  , df=1
parameter F test:         F=0.1849  , p=0.6674  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4220  , p=0.6560  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.8528  , p=0.6529  , df=2
likelihood ratio test: chi2=0.8520  , p=0.6531  , df=2
parameter F test:         F=0.4220  , p=0.6560  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1849  , p=0.6674  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.1861  , p=0.6662  , df=1
likelihood ratio test: chi2=0.1860  , p=0.6662  , df=1
parameter F test:         F=0.1849  , p=0.6674  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4220  , p=0.6560  ,

In [6]:
pd.set_option('display.max_columns',0) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows',0) #replace n with the number of rows you want to see completely

res_table.T.to_excel('上证综指Top100-正向.xlsx')
res_table

,口罩,武汉,湖北,开学,美国,病例,意大利,新冠,肺炎,礼裙,抗疫,钟南山,护士,李兰娟,输入,患者,原来,累计,医疗,方舱,上网,纽约,境外,号房,纽约州,医护人员,感染,出院,特朗普,援鄂,时间,医疗队,确诊,物资,美股,复课,清平乐,冰糖,例新冠,万例,宋仁宗,留学生,呼吸机,称新冠,复工,医护,网课,推迟,重症,医院,组织,女子,回国,鸦杀,返校,触手可及,返京,发热,世卫,本土,接种,初三,破防,火神,2021,大赛,病毒感染,隔离,疫情,医生,狗狗,黄冈,全国,支援,阿水,消杀,防护服,高三,安家,皇后,线上,非洲,小汤山,聚集,龙岭迷,作人,黎语,病毒,病毒检测,延期,N95,基金,治愈,一站,神山,宋凛,男子,毕业生,死亡,泰国
0,1,1,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1
1,1,1,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1,1


## 反向

In [7]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[[column_name, '收盘']], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=46.6679 , p=0.0000  , df=1
likelihood ratio test: chi2=44.5570 , p=0.0000  , df=1
parameter F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=17.7455 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=35.8614 , p=0.0000  , df=2
likelihood ratio test: chi2=34.5951 , p=0.0000  , df=2
parameter F test:         F=17.7455 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=46.6679 , p=0.0000  , df=1
likelihood ratio test: chi2=44.5570 , p=0.0000  , df=1
parameter F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=17.7455 , p=0.0000  

ssr based F test:         F=10.5505 , p=0.0012  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=10.6161 , p=0.0011  , df=1
likelihood ratio test: chi2=10.5016 , p=0.0012  , df=1
parameter F test:         F=10.5505 , p=0.0012  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=7.5773  , p=0.0006  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=15.3128 , p=0.0005  , df=2
likelihood ratio test: chi2=15.0756 , p=0.0005  , df=2
parameter F test:         F=7.5773  , p=0.0006  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=39.9298 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=40.1784 , p=0.0000  , df=1
likelihood ratio test: chi2=38.6007 , p=0.0000  , df=1
parameter F test:         F=39.9298 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=19.6697 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test: 

number of lags (no zero) 1
ssr based F test:         F=30.3190 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=30.5077 , p=0.0000  , df=1
likelihood ratio test: chi2=29.5866 , p=0.0000  , df=1
parameter F test:         F=30.3190 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=13.5334 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=27.3493 , p=0.0000  , df=2
likelihood ratio test: chi2=26.6046 , p=0.0000  , df=2
parameter F test:         F=13.5334 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=30.3190 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=30.5077 , p=0.0000  , df=1
likelihood ratio test: chi2=29.5866 , p=0.0000  , df=1
parameter F test:         F=30.3190 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=13.5334 , p=0.0000  , df_denom=479, df_

ssr based F test:         F=28.2351 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=28.4109 , p=0.0000  , df=1
likelihood ratio test: chi2=27.6099 , p=0.0000  , df=1
parameter F test:         F=28.2351 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=11.3852 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=23.0081 , p=0.0000  , df=2
likelihood ratio test: chi2=22.4780 , p=0.0000  , df=2
parameter F test:         F=11.3852 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=28.2351 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=28.4109 , p=0.0000  , df=1
likelihood ratio test: chi2=27.6099 , p=0.0000  , df=1
parameter F test:         F=28.2351 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=11.3852 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test: 

likelihood ratio test: chi2=4.3478  , p=0.1137  , df=2
parameter F test:         F=2.1611  , p=0.1163  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=27.1624 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=27.3315 , p=0.0000  , df=1
likelihood ratio test: chi2=26.5891 , p=0.0000  , df=1
parameter F test:         F=27.1624 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.4037  , p=0.0001  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=19.0038 , p=0.0001  , df=2
likelihood ratio test: chi2=18.6402 , p=0.0001  , df=2
parameter F test:         F=9.4037  , p=0.0001  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=27.1624 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=27.3315 , p=0.0000  , df=1
likelihood ratio test: chi2=26.5891 , p=0.0000  , df=1
parameter F test:         F=27.1624 , p

...

ssr based chi2 test:   chi2=22.3934 , p=0.0000  , df=1
likelihood ratio test: chi2=21.8918 , p=0.0000  , df=1
parameter F test:         F=22.2549 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=10.1427 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=20.4971 , p=0.0000  , df=2
likelihood ratio test: chi2=20.0749 , p=0.0000  , df=2
parameter F test:         F=10.1427 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=22.2549 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=22.3934 , p=0.0000  , df=1
likelihood ratio test: chi2=21.8918 , p=0.0000  , df=1
parameter F test:         F=22.2549 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=10.1427 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=20.4971 , p=0.0000  , df=2
likelihood ratio test: chi2=20.0749 , p

parameter F test:         F=15.9317 , p=0.0001  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=6.2246  , p=0.0021  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=12.5791 , p=0.0019  , df=2
likelihood ratio test: chi2=12.4184 , p=0.0020  , df=2
parameter F test:         F=6.2246  , p=0.0021  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=28.8817 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=29.0615 , p=0.0000  , df=1
likelihood ratio test: chi2=28.2241 , p=0.0000  , df=1
parameter F test:         F=28.8817 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=12.0087 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=24.2680 , p=0.0000  , df=2
likelihood ratio test: chi2=23.6792 , p=0.0000  , df=2
parameter F test:         F=12.0087 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
nu

ssr based F test:         F=9.6282  , p=0.0020  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=9.6882  , p=0.0019  , df=1
likelihood ratio test: chi2=9.5927  , p=0.0020  , df=1
parameter F test:         F=9.6282  , p=0.0020  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=8.7045  , p=0.0002  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=17.5907 , p=0.0002  , df=2
likelihood ratio test: chi2=17.2785 , p=0.0002  , df=2
parameter F test:         F=8.7045  , p=0.0002  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=14.4764 , p=0.0002  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=14.5665 , p=0.0001  , df=1
likelihood ratio test: chi2=14.3521 , p=0.0002  , df=1
parameter F test:         F=14.4764 , p=0.0002  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=6.1159  , p=0.0024  , df_denom=479, df_num=2
ssr based chi2 test: 

parameter F test:         F=3.5691  , p=0.0289  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=11.4926 , p=0.0008  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=11.5641 , p=0.0007  , df=1
likelihood ratio test: chi2=11.4284 , p=0.0007  , df=1
parameter F test:         F=11.4926 , p=0.0008  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=8.6767  , p=0.0002  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=17.5345 , p=0.0002  , df=2
likelihood ratio test: chi2=17.2244 , p=0.0002  , df=2
parameter F test:         F=8.6767  , p=0.0002  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=11.4926 , p=0.0008  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=11.5641 , p=0.0007  , df=1
likelihood ratio test: chi2=11.4284 , p=0.0007  , df=1
parameter F test:         F=11.4926 , p=0.0008  , df_denom=482, df_num=1

Granger Causality
nu

likelihood ratio test: chi2=8.9003  , p=0.0117  , df=2
parameter F test:         F=4.4449  , p=0.0122  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=16.5137 , p=0.0001  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=16.6165 , p=0.0000  , df=1
likelihood ratio test: chi2=16.3382 , p=0.0001  , df=1
parameter F test:         F=16.5137 , p=0.0001  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.4449  , p=0.0122  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=8.9826  , p=0.0112  , df=2
likelihood ratio test: chi2=8.9003  , p=0.0117  , df=2
parameter F test:         F=4.4449  , p=0.0122  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=10.1536 , p=0.0015  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=10.2168 , p=0.0014  , df=1
likelihood ratio test: chi2=10.1106 , p=0.0015  , df=1
parameter F test:         F=10.1536 , p

In [8]:
pd.set_option('display.max_columns',0) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows',0) #replace n with the number of rows you want to see completely

res_table.T.to_excel('上证综指Top100-反向.xlsx')
res_table

,口罩,武汉,湖北,开学,美国,病例,意大利,新冠,肺炎,礼裙,抗疫,钟南山,护士,李兰娟,输入,患者,原来,累计,医疗,方舱,上网,纽约,境外,号房,纽约州,医护人员,感染,出院,特朗普,援鄂,时间,医疗队,确诊,物资,美股,复课,清平乐,冰糖,例新冠,万例,宋仁宗,留学生,呼吸机,称新冠,复工,医护,网课,推迟,重症,医院,组织,女子,回国,鸦杀,返校,触手可及,返京,发热,世卫,本土,接种,初三,破防,火神,2021,大赛,病毒感染,隔离,疫情,医生,狗狗,黄冈,全国,支援,阿水,消杀,防护服,高三,安家,皇后,线上,非洲,小汤山,聚集,龙岭迷,作人,黎语,病毒,病毒检测,延期,N95,基金,治愈,一站,神山,宋凛,男子,毕业生,死亡,泰国
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1


# Top500

In [9]:
df = pd.read_excel("../相关性系数/日维度关键词热度排序总表.xlsx")
df = df.pivot_table(index='日期',columns='关键词',values='系数',aggfunc='sum').fillna(0)
df2 = pd.read_excel("../相关性系数/相关系数排名-上证综指.xlsx")
df = df[list(df2['关键词'][:500])]
df

关键词,口罩,武汉,湖北,开学,美国,病例,意大利,新冠,肺炎,礼裙,抗疫,钟南山,护士,李兰娟,输入,患者,原来,累计,医疗,方舱,上网,纽约,境外,号房,纽约州,医护人员,感染,出院,特朗普,援鄂,时间,医疗队,确诊,物资,美股,复课,清平乐,冰糖,例新冠,万例,宋仁宗,留学生,呼吸机,称新冠,复工,医护,网课,推迟,重症,医院,组织,女子,回国,鸦杀,返校,触手可及,返京,发热,世卫,本土,...,级别,湖北省,等级,抢购,黄蜂,月底,2022,永胜,真切,学士,万人次,先锋,王一楠,大湾,小孩,校外,搞怪,上官,冬奥会,万人,乱象,弘毅,复业,架次,年代,民法典,vlog,停战协定,釜山,奥特曼,陆绎,这些,张乘,全球,重启,异度,已喜,罗马教皇,外籍,无差别,造谣诽谤,教徒,三千,脚踝,送养,申冰,刑拘,英语听力,主张,买过,小敏家,警察,护目镜,投放,新世界,徐坤青,查扣,戈贝尔,赛大,JonyJ
日期,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,0.000000,0.000301,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.029700,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2020-01-02,0.000000,0.011621,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.022819,0.000000,0.000000,0.000000,0.001084,0.0,0.000000,0.024331,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.011259,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.016160,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000329,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2020-01-03,0.000000,0.008366,0.0,0.000000,0.010290,0.000000,0.000000,0.000000,0.008366,0.000000,0.000000,0.000000,0.001766,0.0,0.000000,0.001422,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00084,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.016279,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.012535,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2020-01-04,0.000000,0.002468,0.0,0.000000,0.033193,0.000000,0.000000,0.000000,0.002468,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.009067,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.006097,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.005193,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000448,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.001208,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2020-01-05,0.000000,0.022405,0.0,0.000000,0.017324,0.000000,0.000000,0.000000,0.022405,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0000

In [10]:
df1 = pd.read_csv("../相关性系数/上证指数历史数据.csv")
df1['日期'] = df1['日期'].apply(date_format)
df1 = df1.sort_values(by='日期').set_index('日期')
df1 = df1['收盘']
df3 = pd.merge(df1, df, how='left',left_index=True,right_index=True)

In [11]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[['收盘', column_name]], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=12.3281 , p=0.0004  , df=1
likelihood ratio test: chi2=12.1741 , p=0.0005  , df=1
parameter F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.4973  , p=0.0001  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=19.1928 , p=0.0001  , df=2
likelihood ratio test: chi2=18.8221 , p=0.0001  , df=2
parameter F test:         F=9.4973  , p=0.0001  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=12.3281 , p=0.0004  , df=1
likelihood ratio test: chi2=12.1741 , p=0.0005  , df=1
parameter F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.4973  , p=0.0001  

ssr based chi2 test:   chi2=0.2332  , p=0.6292  , df=1
likelihood ratio test: chi2=0.2331  , p=0.6292  , df=1
parameter F test:         F=0.2318  , p=0.6304  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3099  , p=0.7337  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.6263  , p=0.7311  , df=2
likelihood ratio test: chi2=0.6259  , p=0.7313  , df=2
parameter F test:         F=0.3099  , p=0.7337  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2318  , p=0.6304  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.2332  , p=0.6292  , df=1
likelihood ratio test: chi2=0.2331  , p=0.6292  , df=1
parameter F test:         F=0.2318  , p=0.6304  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3099  , p=0.7337  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.6263  , p=0.7311  , df=2
likelihood ratio test: chi2=0.6259  , p

parameter F test:         F=0.1033  , p=0.7481  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.5212  , p=0.2195  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=3.0742  , p=0.2150  , df=2
likelihood ratio test: chi2=3.0645  , p=0.2160  , df=2
parameter F test:         F=1.5212  , p=0.2195  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.3924  , p=0.2386  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=1.4011  , p=0.2365  , df=1
likelihood ratio test: chi2=1.3991  , p=0.2369  , df=1
parameter F test:         F=1.3924  , p=0.2386  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.9022  , p=0.4063  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.8233  , p=0.4019  , df=2
likelihood ratio test: chi2=1.8199  , p=0.4025  , df=2
parameter F test:         F=0.9022  , p=0.4063  , df_denom=479, df_num=2

Granger Causality
nu

ssr based F test:         F=0.0764  , p=0.7824  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.0768  , p=0.7816  , df=1
likelihood ratio test: chi2=0.0768  , p=0.7816  , df=1
parameter F test:         F=0.0764  , p=0.7824  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0781  , p=0.9249  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.1578  , p=0.9241  , df=2
likelihood ratio test: chi2=0.1578  , p=0.9242  , df=2
parameter F test:         F=0.0781  , p=0.9249  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0764  , p=0.7824  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.0768  , p=0.7816  , df=1
likelihood ratio test: chi2=0.0768  , p=0.7816  , df=1
parameter F test:         F=0.0764  , p=0.7824  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0781  , p=0.9249  , df_denom=479, df_num=2
ssr based chi2 test: 

parameter F test:         F=1.3840  , p=0.2516  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0121  , p=0.9125  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.0122  , p=0.9122  , df=1
likelihood ratio test: chi2=0.0122  , p=0.9122  , df=1
parameter F test:         F=0.0121  , p=0.9125  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.3840  , p=0.2516  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=2.7968  , p=0.2470  , df=2
likelihood ratio test: chi2=2.7888  , p=0.2480  , df=2
parameter F test:         F=1.3840  , p=0.2516  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.3555  , p=0.5513  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.3578  , p=0.5498  , df=1
likelihood ratio test: chi2=0.3576  , p=0.5498  , df=1
parameter F test:         F=0.3555  , p=0.5513  , df_denom=482, df_num=1

Granger Causality
nu

...

parameter F test:         F=0.8497  , p=0.4282  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.5544  , p=0.2131  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=1.5641  , p=0.2111  , df=1
likelihood ratio test: chi2=1.5616  , p=0.2114  , df=1
parameter F test:         F=1.5544  , p=0.2131  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.8497  , p=0.4282  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.7172  , p=0.4238  , df=2
likelihood ratio test: chi2=1.7141  , p=0.4244  , df=2
parameter F test:         F=0.8497  , p=0.4282  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1067  , p=0.7441  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.1074  , p=0.7432  , df=1
likelihood ratio test: chi2=0.1074  , p=0.7432  , df=1
parameter F test:         F=0.1067  , p=0.7441  , df_denom=482, df_num=1

Granger Causality
nu

number of lags (no zero) 1
ssr based F test:         F=0.4808  , p=0.4884  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.4838  , p=0.4867  , df=1
likelihood ratio test: chi2=0.4835  , p=0.4868  , df=1
parameter F test:         F=0.4808  , p=0.4884  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.0355  , p=0.3558  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=2.0926  , p=0.3512  , df=2
likelihood ratio test: chi2=2.0881  , p=0.3520  , df=2
parameter F test:         F=1.0355  , p=0.3558  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.4808  , p=0.4884  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.4838  , p=0.4867  , df=1
likelihood ratio test: chi2=0.4835  , p=0.4868  , df=1
parameter F test:         F=0.4808  , p=0.4884  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.0355  , p=0.3558  , df_denom=479, df_

likelihood ratio test: chi2=1.8429  , p=0.1746  , df=1
parameter F test:         F=1.8350  , p=0.1762  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.9164  , p=0.4006  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.8520  , p=0.3961  , df=2
likelihood ratio test: chi2=1.8485  , p=0.3968  , df=2
parameter F test:         F=0.9164  , p=0.4006  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.8350  , p=0.1762  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=1.8464  , p=0.1742  , df=1
likelihood ratio test: chi2=1.8429  , p=0.1746  , df=1
parameter F test:         F=1.8350  , p=0.1762  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.9164  , p=0.4006  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.8520  , p=0.3961  , df=2
likelihood ratio test: chi2=1.8485  , p=0.3968  , df=2
parameter F test:         F=0.9164  , p

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1855  , p=0.6669  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.1866  , p=0.6657  , df=1
likelihood ratio test: chi2=0.1866  , p=0.6658  , df=1
parameter F test:         F=0.1855  , p=0.6669  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1232  , p=0.8841  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.2489  , p=0.8830  , df=2
likelihood ratio test: chi2=0.2489  , p=0.8830  , df=2
parameter F test:         F=0.1232  , p=0.8841  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1855  , p=0.6669  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.1866  , p=0.6657  , df=1
likelihood ratio test: chi2=0.1866  , p=0.6658  , df=1
parameter F test:         F=0.1855  , p=0.6669  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1232  , p=0.8841  ,

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0147  , p=0.9035  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.0148  , p=0.9031  , df=1
likelihood ratio test: chi2=0.0148  , p=0.9031  , df=1
parameter F test:         F=0.0147  , p=0.9035  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0059  , p=0.9941  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.0119  , p=0.9941  , df=2
likelihood ratio test: chi2=0.0119  , p=0.9941  , df=2
parameter F test:         F=0.0059  , p=0.9941  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0428  , p=0.8361  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.0431  , p=0.8355  , df=1
likelihood ratio test: chi2=0.0431  , p=0.8355  , df=1
parameter F test:         F=0.0428  , p=0.8361  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0478  , p=0.9533  ,

In [12]:
pd.set_option('display.max_columns',0) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows',0) #replace n with the number of rows you want to see completely

res_table.T.to_excel('上证综指Top500-正向.xlsx')
res_table

,口罩,武汉,湖北,开学,美国,病例,意大利,新冠,肺炎,礼裙,抗疫,钟南山,护士,李兰娟,输入,患者,原来,累计,医疗,方舱,上网,纽约,境外,号房,纽约州,医护人员,感染,出院,特朗普,援鄂,时间,医疗队,确诊,物资,美股,复课,清平乐,冰糖,例新冠,万例,宋仁宗,留学生,呼吸机,称新冠,复工,医护,网课,推迟,重症,医院,组织,女子,回国,鸦杀,返校,触手可及,返京,发热,世卫,本土,...,级别,湖北省,等级,抢购,黄蜂,月底,2022,永胜,真切,学士,万人次,先锋,王一楠,大湾,小孩,校外,搞怪,上官,冬奥会,万人,乱象,弘毅,复业,架次,年代,民法典,vlog,停战协定,釜山,奥特曼,陆绎,这些,张乘,全球,重启,异度,已喜,罗马教皇,外籍,无差别,造谣诽谤,教徒,三千,脚踝,送养,申冰,刑拘,英语听力,主张,买过,小敏家,警察,护目镜,投放,新世界,徐坤青,查扣,戈贝尔,赛大,JonyJ
0,1,1,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1,1,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


## 反向

In [13]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[[column_name, '收盘']], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=46.6679 , p=0.0000  , df=1
likelihood ratio test: chi2=44.5570 , p=0.0000  , df=1
parameter F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=17.7455 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=35.8614 , p=0.0000  , df=2
likelihood ratio test: chi2=34.5951 , p=0.0000  , df=2
parameter F test:         F=17.7455 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=46.6679 , p=0.0000  , df=1
likelihood ratio test: chi2=44.5570 , p=0.0000  , df=1
parameter F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=17.7455 , p=0.0000  

ssr based F test:         F=13.8582 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=28.0057 , p=0.0000  , df=2
likelihood ratio test: chi2=27.2254 , p=0.0000  , df=2
parameter F test:         F=13.8582 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=27.8545 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=28.0279 , p=0.0000  , df=1
likelihood ratio test: chi2=27.2479 , p=0.0000  , df=1
parameter F test:         F=27.8545 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=13.8582 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=28.0057 , p=0.0000  , df=2
likelihood ratio test: chi2=27.2254 , p=0.0000  , df=2
parameter F test:         F=13.8582 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=18.3383 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test: 

ssr based F test:         F=18.7577 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=18.8744 , p=0.0000  , df=1
likelihood ratio test: chi2=18.5164 , p=0.0000  , df=1
parameter F test:         F=18.7577 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.4738  , p=0.0001  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=19.1455 , p=0.0001  , df=2
likelihood ratio test: chi2=18.7765 , p=0.0001  , df=2
parameter F test:         F=9.4738  , p=0.0001  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=18.7577 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=18.8744 , p=0.0000  , df=1
likelihood ratio test: chi2=18.5164 , p=0.0000  , df=1
parameter F test:         F=18.7577 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.4738  , p=0.0001  , df_denom=479, df_num=2
ssr based chi2 test: 

parameter F test:         F=22.1701 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=11.0325 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=22.2954 , p=0.0000  , df=2
likelihood ratio test: chi2=21.7971 , p=0.0000  , df=2
parameter F test:         F=11.0325 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=6.5242  , p=0.0109  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=6.5648  , p=0.0104  , df=1
likelihood ratio test: chi2=6.5208  , p=0.0107  , df=1
parameter F test:         F=6.5242  , p=0.0109  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.1611  , p=0.1163  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=4.3673  , p=0.1126  , df=2
likelihood ratio test: chi2=4.3478  , p=0.1137  , df=2
parameter F test:         F=2.1611  , p=0.1163  , df_denom=479, df_num=2

Granger Causality
nu

ssr based F test:         F=41.4072 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=41.6649 , p=0.0000  , df=1
likelihood ratio test: chi2=39.9716 , p=0.0000  , df=1
parameter F test:         F=41.4072 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=23.1022 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=46.6866 , p=0.0000  , df=2
likelihood ratio test: chi2=44.5700 , p=0.0000  , df=2
parameter F test:         F=23.1022 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=12.9535 , p=0.0004  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=13.0341 , p=0.0003  , df=1
likelihood ratio test: chi2=12.8620 , p=0.0003  , df=1
parameter F test:         F=12.9535 , p=0.0004  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.5974  , p=0.0040  , df_denom=479, df_num=2
ssr based chi2 test: 

...

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=6.1449  , p=0.0135  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=6.1832  , p=0.0129  , df=1
likelihood ratio test: chi2=6.1441  , p=0.0132  , df=1
parameter F test:         F=6.1449  , p=0.0135  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=12.6607 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=25.5857 , p=0.0000  , df=2
likelihood ratio test: chi2=24.9323 , p=0.0000  , df=2
parameter F test:         F=12.6607 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=6.1449  , p=0.0135  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=6.1832  , p=0.0129  , df=1
likelihood ratio test: chi2=6.1441  , p=0.0132  , df=1
parameter F test:         F=6.1449  , p=0.0135  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=12.6607 , p=0.0000  ,

ssr based F test:         F=8.2620  , p=0.0003  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=16.6964 , p=0.0002  , df=2
likelihood ratio test: chi2=16.4149 , p=0.0003  , df=2
parameter F test:         F=8.2620  , p=0.0003  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=11.7040 , p=0.0007  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=11.7769 , p=0.0006  , df=1
likelihood ratio test: chi2=11.6362 , p=0.0006  , df=1
parameter F test:         F=11.7040 , p=0.0007  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=8.2620  , p=0.0003  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=16.6964 , p=0.0002  , df=2
likelihood ratio test: chi2=16.4149 , p=0.0003  , df=2
parameter F test:         F=8.2620  , p=0.0003  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=5.6980  , p=0.0174  , df_denom=482, df_num=1
ssr based chi2 test: 

number of lags (no zero) 2
ssr based F test:         F=2.0091  , p=0.1352  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=4.0601  , p=0.1313  , df=2
likelihood ratio test: chi2=4.0431  , p=0.1324  , df=2
parameter F test:         F=2.0091  , p=0.1352  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.4916  , p=0.1151  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=2.5071  , p=0.1133  , df=1
likelihood ratio test: chi2=2.5006  , p=0.1138  , df=1
parameter F test:         F=2.4916  , p=0.1151  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.0091  , p=0.1352  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=4.0601  , p=0.1313  , df=2
likelihood ratio test: chi2=4.0431  , p=0.1324  , df=2
parameter F test:         F=2.0091  , p=0.1352  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.4341  , p=0.0645  , df_denom=482, df_

likelihood ratio test: chi2=7.7141  , p=0.0211  , df=2
parameter F test:         F=3.8478  , p=0.0220  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=7.6772  , p=0.0058  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=7.7250  , p=0.0054  , df=1
likelihood ratio test: chi2=7.6641  , p=0.0056  , df=1
parameter F test:         F=7.6772  , p=0.0058  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.8478  , p=0.0220  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=7.7759  , p=0.0205  , df=2
likelihood ratio test: chi2=7.7141  , p=0.0211  , df=2
parameter F test:         F=3.8478  , p=0.0220  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=4.9985  , p=0.0258  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=5.0296  , p=0.0249  , df=1
likelihood ratio test: chi2=5.0037  , p=0.0253  , df=1
parameter F test:         F=4.9985  , p

likelihood ratio test: chi2=5.9110  , p=0.0150  , df=1
parameter F test:         F=5.9103  , p=0.0154  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.8663  , p=0.0030  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=11.8551 , p=0.0027  , df=2
likelihood ratio test: chi2=11.7122 , p=0.0029  , df=2
parameter F test:         F=5.8663  , p=0.0030  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=7.6944  , p=0.0058  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=7.7423  , p=0.0054  , df=1
likelihood ratio test: chi2=7.6811  , p=0.0056  , df=1
parameter F test:         F=7.6944  , p=0.0058  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.5739  , p=0.0108  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=9.2433  , p=0.0098  , df=2
likelihood ratio test: chi2=9.1562  , p=0.0103  , df=2
parameter F test:         F=4.5739  , p

In [14]:
pd.set_option('display.max_columns',0) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows',0) #replace n with the number of rows you want to see completely

res_table.T.to_excel('上证综指Top500-反向.xlsx')
res_table

,口罩,武汉,湖北,开学,美国,病例,意大利,新冠,肺炎,礼裙,抗疫,钟南山,护士,李兰娟,输入,患者,原来,累计,医疗,方舱,上网,纽约,境外,号房,纽约州,医护人员,感染,出院,特朗普,援鄂,时间,医疗队,确诊,物资,美股,复课,清平乐,冰糖,例新冠,万例,宋仁宗,留学生,呼吸机,称新冠,复工,医护,网课,推迟,重症,医院,组织,女子,回国,鸦杀,返校,触手可及,返京,发热,世卫,本土,...,级别,湖北省,等级,抢购,黄蜂,月底,2022,永胜,真切,学士,万人次,先锋,王一楠,大湾,小孩,校外,搞怪,上官,冬奥会,万人,乱象,弘毅,复业,架次,年代,民法典,vlog,停战协定,釜山,奥特曼,陆绎,这些,张乘,全球,重启,异度,已喜,罗马教皇,外籍,无差别,造谣诽谤,教徒,三千,脚踝,送养,申冰,刑拘,英语听力,主张,买过,小敏家,警察,护目镜,投放,新世界,徐坤青,查扣,戈贝尔,赛大,JonyJ
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0,1,1,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1
